**Данные для следующих задач: Airbnb в Берлине. Не забывайте о преобразовании столбцов к нужному типу данных!**


**Сначала оставьте только те объявления, в которых оценка на основе отзывов выше среднего, а число отзывов в месяц составляет строго меньше трёх. Затем отсортируйте по убыванию две колонки: сначала по числу отзывов в месяц, потом по оценке. В качестве ответа укажите id объявления из первой строки.**

* review_scores_rating – оценка на основе отзывов
* reviews_per_month – число отзывов в месяц
* id – идентификатор объявления

Таблица – **listings**. Наивысшие показатели = сортировка по убыванию.


Ответ: `22476940`


```sql
SELECT 
    id,
    toFloat64OrNull(review_scores_rating) as review_scores_rating,
    reviews_per_month as reviews_per_month
FROM
    listings
where reviews_per_month < 3 and review_scores_rating > (
                                                        select 
                                                            avg(toFloat64OrNull(review_scores_rating)) 
                                                        from 
                                                            listings)
order by reviews_per_month desc, review_scores_rating desc
limit 1
```


**4. Посчитайте среднее расстояние до центра города и выведите идентификаторы объявлений о сдаче отдельных комнат, для которых расстояние оказалось меньше среднего. Результат отсортируйте по убыванию, тем самым выбрав комнату, которая является наиболее удаленной от центра, но при этом расположена ближе, чем остальные комнаты в среднем.** 

* id – идентификатор объявления
* host_id – идентификатор хозяина
* room_type – тип жилья ('Private room')
* latitude – широта
* longitude – долгота
* 52.5200 с.ш., 13.4050 в.д – координаты центра Берлина

**В качестве ответа укажите идентификатор хозяина (host_id), сдающего данную комнату.**

* geoDistance(13.4050, 52.5200, долгота, широта)

Note: задание можно решить как используя подзапрос в WHERE, так и с помощью WITH. При подсчете среднего тип комнаты тоже следует учесть.


Ответ: `163065208`

```sql
SELECT 
    id,
    host_id,
    geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)) as distance
FROM
    listings
where room_type = 'Private room' and distance < (
    select 
        avg(geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)))
    from 
        listings
    where room_type = 'Private room'
    )
order by distance desc
limit 100
```

**5. Представим, что вы планируете снять жилье в Берлине на 7 дней, используя более хитрые фильтры, чем предлагаются на сайте.**

**В этой задаче можно потренироваться в написании подзапросов, но задание можно решить и без них.**

**Отберите объявления из таблицы listings, которые:**

* находятся на расстоянии от центра меньше среднего (здесь нам пригодится запрос из предыдущего задания)
* обойдутся дешевле 100$ в день (price с учетом cleaning_fee, который добавляется к общей сумме за неделю, т.е его нужно делить на кол-во дней)
* имеют последние отзывы (last_review), начиная с 1 сентября 2018 года
* имеют WiFi в списке удобств (amenities)

**Отсортируйте полученные значения по убыванию review_scores_rating (не забудьте перевести строку к численному виду) и в качестве ответа укажите host_id из первой строки.** 

Hint: [документация](https://clickhouse.tech/docs/ru/sql-reference/functions/string-search-functions/#function-multisearchany) по функции **multiSearchAnyCaseInsensitive(колонка, ['искомая_подстрока'])**, которая ищет заданную подстроку в строке вне зависимости от регистра и выдает 0 в случае, если подстрока не была найдена.



Ответ: `39740677`

```sql
SELECT 
    id,
    host_id,
    toFloat32OrNull(review_scores_rating) as review_scores_rating,
    geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude)) as distance,
    last_review,
    toFloat32OrNull(replaceRegexpAll(cleaning_fee, '[$,]', '')) / 7 + toFloat32OrNull(replaceRegexpAll(price, '[$,]', '')) as price_per_day
FROM listings
WHERE last_review > '2018-09-01' and multiSearchAnyCaseInsensitive(amenities, ['WIFI']) = 1 and distance < (
SELECT avg(geoDistance(13.4050, 52.5200, toFloat32OrNull(longitude), toFloat32OrNull(latitude))) from listings
) and price_per_day < 100
order by review_scores_rating desc
LIMIT 100
```

**6. Давайте найдем в таблице calendar_summary те доступные (available='t') объявления, у которых число отзывов от уникальных пользователей в таблице reviews выше среднего.**

**Для этого с помощью конструкции WITH посчитайте среднее число уникальных reviewer_id из таблицы reviews на каждое жильё, потом проведите джойн таблиц calendar_summary и reviews по полю listing_id (при этом из таблицы calendar_summary должны быть отобраны уникальные listing_id, отфильтрованные по правилу available='t'). Результат отфильтруйте так, чтобы остались только записи, у которых число отзывов от уникальных людей выше среднего.**

**Отсортируйте результат по возрастанию listing_id и в качестве ответа впишите количество отзывов от уникальных пользователей из первой строки.**

Hint: для решения проблем со вложенными агрегационными функциями (и агрегационными функциями там, где их не ждут) как раз очень помогут подзапросы.


Ответ: `118`

```sql
WITH (
    SELECT 
        AVG(count_reviewer_id) 
    FROM
        (SELECT 
            listing_id, COUNT(DISTINCT reviewer_id) AS count_reviewer_id
        FROM 
            reviews AS re
        JOIN 
        (SELECT 
            DISTINCT listing_id
        FROM 
            calendar_summary
        WHERE 
            available = 't'
        ) AS cs
        ON cs.listing_id = re.listing_id
        GROUP BY  listing_id
        ORDER BY listing_id
        )) AS avg_reviewers

SELECT 
    listing_id, COUNT(DISTINCT reviewer_id) AS count_reviewers 
FROM 
    reviews AS re
JOIN (
    SELECT 
        DISTINCT listing_id 
    FROM 
        calendar_summary
    WHERE 
        available = 't'
    ) AS cs
ON cs.listing_id = re.listing_id 
GROUP BY listing_id
HAVING count_reviewers > avg_reviewers
ORDER BY listing_id
LIMIT 100
```

---------------
**Возвращаемся к данными мобильного приложения!**

**installs** — содержит данные об установках приложения по дням.

* DeviceID — идентификатор устройства, на которое было установлено приложение;
* InstallationDate — дата установки приложения;
* InstallCost — цена установки приложения в рублях;
* Platform — платформа, на которой было установлено приложение (iOS/ Android);
* Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

**events** — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

* DeviceID — идентификатор устройства, на котором используется приложение;
* AppPlatform — платформа, на которой используется приложение (iOS/ Android);
* EventDate — дата, за которую собрана статистика;
* events — количество просмотров всех товаров за этот день у этого DeviceID.

**checks** — содержит данные о покупках пользователей в приложении по дням

* UserID — идентификатор пользователя;
* Rub — суммарный чек пользователя на дату;
* BuyDate — дата, за которую собрана статистика.

**devices** – чтобы просмотры и установки можно было объединить с покупками

* DeviceID — идентификатор устройства;
* UserID — идентификатор пользователя.

**7. С помощью оператора CASE в SQL можно составлять условные конструкции, выполнять проверку условий и возвращать результат в зависимости от их выполнения. Самих условий под оператором WHEN может быть сколько угодно, конструкцию необходимо завершить оператором END. Структура выглядит следующим образом:**

    
* **WHERE price > 1000 THEN 'ok' ELSE 'not ok' END**
* **CASE WHEN price>1000 THEN 'ok' WHEN price<500 THEN 'not ok' ELSE 'normal' END**
* CASE WHEN price > 1000 THEN 'ok' ELSE 'not ok'


**8. Теперь задача! Используйте таблицу checks и разделите всех покупателей на сегменты:**

NB! Правые границы берутся не включительно, например, чек в 10 рублей будет относиться к сегменту С

* А — средний чек покупателя менее 5 ₽
* B — средний чек покупателя от 5-10 ₽
* C — средний чек покупателя от 10-20 ₽
* D — средний чек покупателя от 20 ₽

**Отсортируйте результирующую таблицу по возрастанию UserID и укажите сегмент четвертого пользователя.**


* C
* D
* A
* B


```sql
SELECT 
    UserID, 
    CASE
        WHEN AVG(Rub) < 5 THEN 'A'
        WHEN AVG(Rub) >= 5 and AVG(Rub) < 10 THEN 'B'
        WHEN AVG(Rub) >= 10 and AVG(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment
FROM 
    checks
GROUP BY UserID
ORDER BY UserID
LIMIT 100
```

**9. Используйте предыдущий запрос как подзапрос и посчитайте, сколько клиентов приходится на каждый сегмент и сколько доходов он приносит. Отсортируйте результат по убыванию суммы доходов на сегмент и в качестве ответа укажите наибольшую сумму.**

Ответ: `250551286`

```sql
SELECT
    segment, count(UserID), SUM(Rub) AS Revenue
FROM
    checks
JOIN 
    (SELECT 
        UserID,
    CASE
        WHEN AVG(Rub) < 5 THEN 'A'
        WHEN AVG(Rub) >= 5 AND AVG(Rub) < 10 THEN 'B'
        WHEN AVG(Rub) >= 10 AND AVG(Rub) < 20 THEN 'C'
        ELSE 'D'
    END AS segment
    FROM 
        checks
    GROUP BY UserID
    ORDER BY UserID
    ) AS segments
ON segments.UserID = checks.UserID
GROUP BY segment
ORDER BY Revenue DESC
```

**10. Вернемся к таблице AirBnb. Предположим, что в выборе жилья нас интересует только два параметра: наличие кухни (kitchen) и гибкой системы отмены (flexible), причем первый в приоритете.**

**Создайте с помощью оператора CASE колонку с обозначением группы, в которую попадает жилье из таблицы listings:**

* 'good', если в удобствах (amenities) присутствует кухня и система отмены (cancellation_policy) гибкая
* 'ok', если в удобствах есть кухня, но система отмены не гибкая
* 'not ok' во всех остальных случаях

**Результат отсортируйте по новой колонке по возрастанию, установите ограничение в 5 строк, в качестве ответа укажите host_id первой строки.**

**Обратите внимание, что cancellation_policy - это отдельная колонка, по ней необходимо смотреть систему отмены**


Ответ: `27607502`


```sql
SELECT 
    host_id,
    CASE
        WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])!=0 AND cancellation_policy = 'flexible' THEN 'good'
        WHEN multiSearchAnyCaseInsensitive(amenities, ['kitchen'])!=0 AND cancellation_policy != 'flexible' THEN 'ok'
        ELSE 'not ok'
    END AS category
FROM 
    listings
ORDER BY category
LIMIT 5
```

------------------
## Часть 2.

**1. Сопоставьте операторы с соответствующим описанием**

INSERT INTO - **Записать данные в таблицу**

DROP COLUMN - **Удалить столбец**

DROP TABLE - **Удалить таблицу**

ADD COLUMN - **Добавить столбец**

DELETE WHERE - **Удалить строки, где выполняется условие**


**2. Напишите запрос для создания таблицы со следующими параметрами, также подобрав подходящий тип данных.**

Название таблицы: **reviews**

База данных: **test**

**Столбцы:**

- listing_id – идентификатор объявления, может быть только положительным и целым числом, 32-битный тип данных

- id – идентификатор хозяина, может быть только положительным и целым числом, 32-битный тип данных

- created_at – дата со временем (2020-01-01 00:00:00), часовой пояс – 'Europe/Moscow'

- reviewer_id – идентификатор ревьюера, может быть только положительным и целым числом, 32-битный тип данных

- reviewer_name – имя того, кто оставил отзыв

- comments - текст отзыва

Движок: **MergeTree**

Сортировка: **listing_id, id**

```sql
CREATE TABLE test.reviews
    (listing_id UInt32,
    id UInt32,
    create_at DateTime('Europe/Moscow'),
    reviewer_id UInt32,
    reviewer_name String,
    comments String)
    ENGINE = MergeTree
ORDER BY (listing_id, id)
```

**3. К вам пришел коллега с новостями: оказывается, в поле date будет записываться только дата, без времени, поэтому нужно изменить тип данных!**

**Напишите запрос, который модифицирует тип данных, и введите его в поле ниже без кавычек и лишних пробелов. Не забудьте указать базу данных test перед названием таблицы!**


```sql
ALTER TABLE test.reviews MODIFY COLUMN date Date
```

**4. Предположим, ваш коллега вставил данные, но что-то перепутал. Часть строк с комментариями осталась совершенно пустой!** 

**Напишите запрос, который удалит из таблицы test.reviews те строки, где в столбце comments встречаются пустые значения (''). Введите его в поле ниже без кавычек и лишних пробелов.**


```sql
ALTER TABLE test.reviews_zxc DELETE WHERE comments = ''
```

**5. С помощью какого запроса можно создать обычное представление над таблицей test.reviews, которое будет содержать все записи из test.reviews, сгруппированные по reviewer_id с подсчитанным количеством отзывов (id) на каждого пользователя?**

* **CREATE VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count FROM test.reviews GROUP BY reviewer_id)**
* CREATE VIEW test.reviews_number AS (SELECT reviewer_id, SUM(id) reviews_count GROUP BY reviewer_id)
* ALTER TABLE test.reviews_number UPDATE id = COUNT(id)
* CREATE MATERIALIZED VIEW test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count GROUP BY reviewer_id)
* CREATE TABLE IF NOT EXISTS test.reviews_number AS (SELECT reviewer_id, COUNT(id) reviews_count GROUP BY reviewer_id)


**6. С помощью какого запроса можно создать новый столбец reviewer_score в таблице reviews после столбца reviewer_name?**

* ALTER TABLE test.reviews AFTER reviewer_name ADD COLUMN reviewer_score UInt8
* **ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8 AFTER reviewer_name**
* ALTER TABLE test.reviews ADD COLUMN reviewer_score UInt8
* ALTER TABLE test.reviews AFTER reviewer_name INSERT reviewer_score UInt8
* ALTER TABLE test.reviews INSERT COLUMN reviewer_score UInt8 AFTER reviewer_name


**7. Напишите запрос для добавления в таблицу test.reviews колонки price после колонки comments, которая может быть числом с плавающей точкой, 32-битный тип данных.**


```sql
ALTER TABLE test.reviews ADD COLUMN price Float32 AFTER comments
```

**8. Напишите запрос, который удвоит price для всех строк с датой (date) после 2019-01-01.**

**Обратите внимание, что для сравнения с датой в формате '2019-01-01' можно не преобразовывать данные в колонке date.**

Note: ответ введите без лишних пробелов между операциями умножения и сравнения и знака '='.


```sql
ALTER TABLE test.reviews UPDATE price=price*2 WHERE date>'2019-01-01'
```